In [3]:
import yfinance as yf
yf.download("360ONE.NS").reset_index()

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-09-19,302.500000,317.625000,302.500000,317.625000,266.247467,7109860
1,2019-09-20,332.250000,333.500000,332.250000,333.500000,279.554626,2760648
2,2019-09-23,350.174988,350.174988,350.174988,350.174988,293.532379,34096
3,2019-09-24,367.674988,367.674988,367.674988,367.674988,308.201660,2496652
4,2019-09-25,386.049988,386.049988,349.299988,351.237488,294.423035,530580
...,...,...,...,...,...,...,...
1072,2024-01-16,660.000000,668.200012,654.549988,661.599976,661.599976,141899
1073,2024-01-17,655.000000,658.450012,637.650024,646.549988,646.549988,266993
1074,2024-01-18,646.549988,654.500000,638.099976,643.049988,643.049988,229384
1075,2024-01-19,650.000000,659.000000,626.049988,628.750000,628.750000,277493


In [4]:
df_test.iloc[-1]['Date']

NameError: name 'df_test' is not defined

In [ ]:
def plot_candlesticks(df_test):
    fig = go.Figure()

    fig.add_trace(
        go.Candlestick(
            x=df_test['Date'],
            open=df_test['Open'],
            high=df_test['High'],
            low=df_test['Low'],
            close=df_test['Close'],
            showlegend=False,
        )
    )

    fig.add_trace(
        go.Line(x=df_test['Date'], y=df_test[f'target_ma'], name=f'Target SMA')
    )

    df_pattern = (
        df_test[df_test['pred']]
        .groupby((~df_test['pred']).cumsum())
        ['Date']
        .agg(['first', 'last'])
    )

    last_date_included = False
    for idx, row in df_pattern.iterrows():
        fig.add_vrect(
            x0=row['first'],
            x1=row['last'],
            line_width=0,
            fillcolor='green',
            opacity=0.2,
        )
        print(f"first : {row['first']} last: {row['last']}")
        if df_test.iloc[-1]['Date'] >= row['first'] and df_test.iloc[-1]['Date'] <= row['last'] and (row['last']!=row['first']):
            last_date_included = True
        firstgreenflag=True
    
    print( last_date_included)
    # Check if the last entry has pred as True and is not included in any green fill
    if df_test.iloc[-1]['pred'] and not last_date_included:
        fig.add_vline(
            x=df_test.iloc[-1]['Date'], 
            line_width=5, 
            line_color='green'
        )

    fig.update_layout(
        xaxis_rangeslider_visible=True,
        xaxis_title='Date',
        yaxis_title='Price ($)',
        title='Classified Results on SPY',
        width=1000,
        height=700,
    )
    
    fig.show()

In [ ]:
import streamlit as st
import pandas as pd
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import datetime
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np
from sklearn.metrics import roc_curve, auc
import plotly.graph_objects as go
import pickle


df = yf.download("NVDA").reset_index()

In [ ]:
# Feature deriving
# Distance from the moving averages
def create_feature_cols(df):
    for m in [10, 20, 30, 50, 100]:
        df[f'feat_dist_from_ma_{m}'] = df['Close']/df['Close'].rolling(m).mean()-1

    # Distance from n day max/min
    for m in [3, 5, 10, 15, 20, 30, 50, 100]:
        df[f'feat_dist_from_max_{m}'] = df['Close']/df['High'].rolling(m).max()-1
        df[f'feat_dist_from_min_{m}'] = df['Close']/df['Low'].rolling(m).min()-1

    # Price distance
    for m in [1, 2, 3, 4, 5, 10, 15, 20, 30, 50, 100]:
        df[f'feat_price_dist_{m}'] = df['Close']/df['Close'].shift(m)-1

    # Target = if the price above the 20 ma in 5 days time
    df['target_ma'] = df['Close'].rolling(20).mean()
    df['price_above_ma'] = df['Close'] > df['target_ma']
    return df
df = create_feature_cols(df)
df['target'] = df['price_above_ma'].astype(int).shift(-5)
df = df.dropna()

In [ ]:
df.tail(10)

In [ ]:
feat_cols = [col for col in df.columns if 'feat' in col]
train_until = '2019-01-01'
test_until = '2023-12-10'

x_train = df[df['Date'] < train_until][feat_cols]
y_train = df[df['Date'] < train_until]['target']

# df2 = yf.download("NVDA").reset_index()
# df2 = create_feature_cols(df2)
x_test = df[(df['Date'] >= train_until) & (df['Date'] <= test_until)][feat_cols]
y_test = df[(df['Date'] >= train_until) & (df['Date'] <= test_until)]['target']

clf = RandomForestClassifier(
n_estimators=100,
max_depth=3,
random_state=42,
class_weight='balanced',
)

clf.fit(x_train, y_train)

y_train_pred = clf.predict(x_train)
y_test_pred = clf.predict(x_test)

# Calculate accuracy and precision for training data
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred)

# Calculate accuracy and precision for test data
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred)

In [ ]:
df_test = df[(df['Date'] >= train_until) & (df['Date'] <= test_until) ].reset_index(drop=True)
df_test['pred_prob'] = clf.predict_proba(x_test)[:, 1]
df_test['pred'] = df_test['pred_prob'] > 0.5

In [ ]:
df_test

In [ ]:
df_test = df2[df2['Date'] > test_until].reset_index(drop=True)
df_test['pred_prob'] = clf.predict_proba(df_test[x_test.columns])[:, 1]
df_test['pred'] = df_test['pred_prob'] > 0.5

In [ ]:
df_test

In [ ]:
plot_candlesticks(df_test)

In [ ]:
x_test.columns

In [ ]:
import pickle
from datetime import datetime, timedelta
# test for check_if_today_starts_with_vertical_green_overlay

df = yf.download("AMD").reset_index()

def check_if_today_starts_with_vertical_green_overlay(df_test):
    df_pattern = (
        df_test[df_test['pred']]
        .groupby((~df_test['pred']).cumsum())
        ['Date']
        .agg(['first', 'last'])
    )
    
    last_date_included = False
    for idx, row in df_pattern.iterrows():
        if df_test.iloc[-1]['Date'] >= row['first'] and df_test.iloc[-1]['Date'] <= row['last'] and (row['last']!=row['first']):
            last_date_included = True
    
    # Check if the last entry has pred as True and is not included in any green fill
    if df_test.iloc[-1]['pred'] and not last_date_included:
        return True

    return False

def create_feature_cols(df):
    for m in [10, 20, 30, 50, 100]:
        df[f'feat_dist_from_ma_{m}'] = df['Close']/df['Close'].rolling(m).mean()-1

    # Distance from n day max/min
    for m in [3, 5, 10, 15, 20, 30, 50, 100]:
        df[f'feat_dist_from_max_{m}'] = df['Close']/df['High'].rolling(m).max()-1
        df[f'feat_dist_from_min_{m}'] = df['Close']/df['Low'].rolling(m).min()-1

    # Price distance
    for m in [1, 2, 3, 4, 5, 10, 15, 20, 30, 50, 100]:
        df[f'feat_price_dist_{m}'] = df['Close']/df['Close'].shift(m)-1

    # Target = if the price above the 20 ma in 5 days time
    df['target_ma'] = df['Close'].rolling(20).mean()
    df['price_above_ma'] = df['Close'] > df['target_ma']
    return df

def test_model(selected_option, last_n_days):
    with open(f"../models/{selected_option}_model.pkl", "rb") as f:
        clf = pickle.load(f)

    current_date = datetime.now() - timedelta(days=2)
    # Subtract last_n_days days from the current date
    test_until = current_date - timedelta(days=int(last_n_days))
    # Format the date as a string if necessary
    test_until = test_until.strftime('%Y-%m-%d')

    df2 = yf.download(selected_option).reset_index()
    df2 = create_feature_cols(df2)

    # show prediction on last last_n_days days
    df_test = df2[df2['Date'] > test_until].reset_index(drop=True)
    df_test['pred_prob'] = clf.predict_proba(df_test[['feat_dist_from_ma_10', 'feat_dist_from_ma_20', 'feat_dist_from_ma_30',
    'feat_dist_from_ma_50', 'feat_dist_from_ma_100', 'feat_dist_from_max_3',
    'feat_dist_from_min_3', 'feat_dist_from_max_5', 'feat_dist_from_min_5',
    'feat_dist_from_max_10', 'feat_dist_from_min_10',
    'feat_dist_from_max_15', 'feat_dist_from_min_15',
    'feat_dist_from_max_20', 'feat_dist_from_min_20',
    'feat_dist_from_max_30', 'feat_dist_from_min_30',
    'feat_dist_from_max_50', 'feat_dist_from_min_50',
    'feat_dist_from_max_100', 'feat_dist_from_min_100', 'feat_price_dist_1',
    'feat_price_dist_2', 'feat_price_dist_3', 'feat_price_dist_4',
    'feat_price_dist_5', 'feat_price_dist_10', 'feat_price_dist_15',
    'feat_price_dist_20', 'feat_price_dist_30', 'feat_price_dist_50',
    'feat_price_dist_100']])[:, 1]
    df_test['pred'] = df_test['pred_prob'] > 0.5

    return df_test

df_test = test_model("AMD", 30)

# at these dates it showed green overlay starting bar during testing, I'm setting these so anytime in the future i can make sure
# the function is working correctly
start_date = '2023-12-11'
end_date = '2024-01-08'

# Filter the DataFrame
df_test = df_test[(df_test['Date'] >= start_date) & (df_test['Date'] <= end_date)]
check_if_today_starts_with_vertical_green_overlay(df_test)

In [ ]:
yf.download("AMD", end=datetime.now().strftime('%Y-%m-%d')).reset_index().columns

In [ ]:
import pandas as pd

def to_pascal_case(s):
    return ''.join(word.capitalize() for word in s.replace('_', ' ').split())

file_paths = [
    "../Quote-Equity-SBIN-EQ-11-12-2022-to-11-01-2023.csv",
    "../Quote-Equity-SBIN-EQ-11-12-2023-to-11-01-2024.csv"
]

dataframes = [pd.read_csv(file) for file in file_paths]

df = pd.concat(dataframes, ignore_index=True)

df.columns = [to_pascal_case(col) for col in df.columns]

df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
df['Adj Close'] = df['Close']



In [ ]:
output

In [ ]:
from NSEDownload import stocks

# Gets data without adjustment for events
df = stocks.get_data(stock_symbol="AAATECH", start_date='15-9-2021', end_date='15-1-2023')

df['Date'] = df.index
df = df [['Date', 'Open Price', 'High Price', 'Low Price', 'Close Price', 'Total Traded Quantity']]
df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
df.reset_index(drop=True)

In [ ]:
import pandas as pd
pd.read_csv("../cache/AAATECH.csv")

In [ ]:
df.reset_index()